This notebook assumes that you have downloaded the [high-resolution jpeg](https://opg.optica.org/viewmedia.cfm?uri=optica-7-12-1805&figure=optica-7-12-1805-g001&imagetype=full) version
of figure 1 in [Inverse-designed photon extractors for optically addressable
defect qubits](https://opg.optica.org/optica/fulltext.cfm?uri=optica-7-12-1805&id=444947).

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as onp
from scipy import ndimage

# Download the high-resolution figure from link below and set `path` to point to the image.
# https://opg.optica.org/viewmedia.cfm?uri=optica-7-12-1805&figure=optica-7-12-1805-g001&imagetype=full
path = "devices.jpg"

# Load image.
im = onp.asarray(Image.open(path))

# Convert to grayscale and normalize.
im = onp.sum(im.astype(float), axis=-1)
im -= onp.amin(im)
im /= onp.amax(im)

# Place a white rectangle to mask an arrow that slightly overlaps with the design
# in subplot (a).
im[200:300, 900:970] = 1.0

plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
ax.imshow(im)

In [ ]:
# Define some functions that extract the designs from an array.

def extract_design(arr, approximate_coords, approximate_shape, output_dim, zoom_factor=5, atol=0.05):
    """Extract a square design from an array.
    
    Args:
        arr: The array from which to extract the design.
        approximate_coords: The approximate center of the design.
        approximate_shape: The approximate shape of the design; should be larger than the
            actual shape to allow trimming.
        output_dim: Specifies the size of the output design: `(output_dim, output_dim)`.
        zoom_factor: Specifies the resolution of the intermediate design produced during
            the design extraction. It is larger than the output design by `zoom_factor`.
        atol: Absolute tolerance for finding edges of the design.

    Returns:
        If the extraction is successful, the output design, with shape `(output_dim,
        output_dim)`. If not succesful, the roughly-cropped design is directly returned.
    """
    i, j = approximate_coords
    h, w = approximate_shape
    arr = arr[
        max(0, i - h // 2):min(i + h // 2, arr.shape[0]),
        max(0, j - w // 2):min(j + w // 2, arr.shape[1]),
    ]
    
    # Estimate the background value.
    border_values = onp.concatenate([arr[:, 0], arr[:, -1], arr[0, :], arr[-1, :]])
    background = onp.median(border_values)
    if not onp.allclose(border_values, background, atol=atol):
        print(
            "WARNING: border values for coords {approximate_coords} are not all "
            "background, returning approximately cropped array!"
        )
        return arr

    is_background = onp.isclose(arr, background, atol=atol)

    # Determine the bounding box for the design.
    i, j = onp.nonzero(~is_background)
    ilo = onp.amin(i)
    ihi = onp.amax(i) + 1
    jlo = onp.amin(j)
    jhi = onp.amax(j) + 1
    
    arr = arr[ilo:ihi, jlo:jhi]
    arr -= onp.amin(arr)
    arr /= onp.amax(arr)

    if not arr.shape[0] == arr.shape[1]:
        print(
            f"WARNING: cropped array for coords {approximate_coords} is not square, got "
            f"shape {arr.shape}. Padding with background."
        )
        pad_width = (
            (0, 0 if arr.shape[0] > arr.shape[1] else 1),
            (0, 0 if arr.shape[1] > arr.shape[0] else 1),
        )
        arr = onp.pad(arr, pad_width, constant_values=background)
        assert arr.shape[0] == arr.shape[1]

    arr_zoom_factor = zoom_factor * output_dim / arr.shape[0]
    arr = ndimage.zoom(arr, arr_zoom_factor)
    arr = (arr < 0.5).astype(float)
    return box_downsample(arr, zoom_factor)


def box_downsample(arr, factor):
    """Downsample `arr` by `factor` using the box downsampling method."""
    arr = arr.reshape((arr.shape[0] // factor, factor, arr.shape[1] // factor, factor))
    return onp.mean(arr, axis=(1, 3))

In [ ]:
APPROXIMATE_SHAPE = (490, 490)

NAMES_AND_COORDS = [
    ("device1", (250, 1200)),
    ("device2", (2220, 380)),
    ("device3", (2220, 960)),
    ("device4", (2220, 1541)),
    ("device5", (2220, 2120)),
]


plt.figure(figsize=(15, 3))

for i, (name, coords) in enumerate(NAMES_AND_COORDS):
    design = extract_design(im, approximate_coords=coords, approximate_shape=APPROXIMATE_SHAPE, output_dim=300)

    onp.savetxt(f"{name}.csv", design, fmt="%.2f", delimiter=",")

    ax = plt.subplot(1, 5, i + 1)
    ax.imshow(design)
    ax.axis(False)